In [2]:
# !pip install firebase-admin
# !pip install google-cloud-storage
# !pip install python-dotenv
# !pip install pandas

In [3]:
import firebase_admin
from firebase_admin import credentials, storage
from firebase_admin import db
import cloudinary
import cloudinary.api
import cloudinary.uploader
from cloudinary.utils import cloudinary_url
import pandas as pd
from tqdm.auto import tqdm
import os
import dotenv
dotenv.load_dotenv()

True

## Firebase Init

In [5]:
service_account_info ={
    "type": os.getenv('FIREBASE_TYPE'),
    "project_id": os.getenv('FIREBASE_PROJECT_ID'),
    "private_key_id": os.getenv('FIREBASE_PRIVATE_KEY_ID'),
    "private_key": os.getenv('FIREBASE_PRIVATE_KEY'),
    "client_email": os.getenv('FIREBASE_CLIENT_EMAIL'),
    "client_id": os.getenv('FIREBASE_CLIENT_ID'),
    "auth_uri": os.getenv('FIREBASE_AUTH_URI'),
    "token_uri": os.getenv('FIREBASE_TOKEN_URI'),
    "auth_provider_x509_cert_url": os.getenv('FIREBASE_AUTH_PROVIDER_X509_CERT_URL'),
    "client_x509_cert_url": os.getenv('FIREBASE_CLIENT_X509_CERT_URL'),
    "universe_domain": os.getenv('FIREBASE_UNIVERSE_DOMAIN')
  }

realtime_db = {
    "realtime_db_url": os.getenv("FIREBASE_DATABASE_URL")    
}

In [6]:
# Check if the default app is already initialized
if not firebase_admin._apps:
    cred = credentials.Certificate(service_account_info)
    firebase_admin.initialize_app(cred, {
        # 'storageBucket': 'coffee-shop-app-a4152.appspot.com',
        'databaseURL': realtime_db["realtime_db_url"]
    })
    print("App Initialized")
else:
    print("Firebase app is already initialized.")

App Initialized


In [7]:
print(realtime_db["realtime_db_url"])

https://cs-chatbot-app-default-rtdb.europe-west1.firebasedatabase.app


In [8]:
# bucket = storage.bucket()

## Cloudinary Init

In [10]:
# Cloudinary Configuration
cloudinary.config( 
    cloud_name = os.getenv('CLOUDINARY_NAME'), 
    api_key = os.getenv('CLOUDINARY_API_KEY'),  
    api_secret = os.getenv('CLOUDINARY_SECRET_KEY'),
    secure=True
)

# Upload Data

In [12]:
image_folder_path = './products/images/'

In [13]:
products_collection = db.reference('products')

In [14]:
df = pd.read_json('products/products.jsonl', lines=True)
df.head(2)

,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",4.50,4.7,cappuccino.jpg
1,Jumbo Savory Scone,Bakery,"Deliciously flaky and buttery, this jumbo savo...","[Flour, Butter, Cheese, Herbs, Baking Powder, ...",3.25,4.3,SavoryScone.webp


### Upload Images and Collection to Firebase Storage and RealTime Database

In [16]:
# # Upload images to firebase storage
# def upload_image(bucket, image_path):
#     image_name = image_path.split('/')[-1]
#     blob = bucket.blob(f'product_images/{image_name}')
#     # Upload image
#     blob.upload_from_filename(image_path)
#     # Make the image publicly accessible and get its URL
#     blob.make_public()
#     return blob.public_url

In [17]:
# for index, row in df.iterrows():
#     print(index, row['name'])
    
#     image_path = os.path.join(image_folder_path, row['image_path'])
    
#     image_url = upload_image(bucket, image_path)
#     product_data = row.to_dict()
#     product_data.pop('image_path')
#     product_data['image_url'] = image_url
    
#     # Add to Firestore
#     products_collection.push().set(product_data)
    

### Upload Images to `Cloudinary` and Collection to Firebase `RealTime Database`

In [19]:
def clear_existing_data():
    """Clear existing data from both Firebase and Cloudinary"""
    print("\n🗑️ Clearing existing data...")
    
    # Clear Firebase collection
    try:
        print("Clearing Firebase collection...", end=" ")
        products_collection.delete()
        print("✅")
    except Exception as e:
        print(f"\n❌ Error clearing Firebase: {e}")
        
    # Clear Cloudinary folder
    try:
        print("Clearing Cloudinary products folder...", end=" ")
        # Get all images in the products folder
        result = cloudinary.api.resources(
            type='upload',
            prefix='products/',
            max_results=500
        )
        
        # Delete each image
        for resource in result.get('resources', []):
            cloudinary.uploader.destroy(resource['public_id'])
        print("✅")
    except Exception as e:
        print(f"\n❌ Error clearing Cloudinary: {e}")

def upload_image_to_cloudinary(image_path):
    """Upload image to Cloudinary and return the secure URL"""
    try:
        image_name = os.path.splitext(os.path.basename(image_path))[0]
        upload_result = cloudinary.uploader.upload(
            image_path,
            public_id=f"product_images/{image_name}",
            folder="products"
        )
        return upload_result["secure_url"]
    except Exception as e:
        print(f"\n❌ Error uploading {image_path}: {e}")
        return None

def upload_products():
    """Upload all products to Firebase and Cloudinary"""
    # First clear existing data
    clear_existing_data()
    
    print("\n📤 Starting fresh upload process...")
    total_products = len(df)
    successful_uploads = 0
    failed_uploads = 0
    
    # Create progress bar
    pbar = tqdm(total=total_products, desc="Uploading products")
    
    for index, row in df.iterrows():
        try:
            pbar.set_description(f"Processing: {row['name']}")
            
            # Upload image to Cloudinary
            image_path = os.path.join(image_folder_path, row['image_path'])
            image_url = upload_image_to_cloudinary(image_path)
            
            if not image_url:
                failed_uploads += 1
                pbar.update(1)
                continue
            
            # Prepare and upload product data to Firebase
            product_data = row.to_dict()
            product_data.pop('image_path', None)  # Remove local image path
            product_data['image_url'] = image_url  # Add Cloudinary URL
            
            # Upload to Firebase
            products_collection.push().set(product_data)
            successful_uploads += 1
            
        except Exception as e:
            print(f"\n❌ Error processing {row.get('name', 'unknown')}: {e}")
            failed_uploads += 1
        finally:
            pbar.update(1)
    
    # Close progress bar
    pbar.close()
    
    # Print final statistics
    print("\n" + "="*40)
    print("        Upload Summary        ")
    print("="*40)
    print(f"Total products      : {total_products}")
    print(f"Successful uploads  : {successful_uploads}")
    print(f"Failed uploads     : {failed_uploads}")
    print(f"Success rate       : {(successful_uploads/total_products)*100:.1f}%")
    print("="*40)

In [20]:
# Ask for confirmation before proceeding
print("\n⚠️  Warning: This will delete all existing products and images!")
confirmation = input("Are you sure you want to proceed? (yes/no): ")

if confirmation.lower() == 'yes':
    upload_products()
else:
    print("\nOperation cancelled.")



⚠️  Warning: This will delete all existing products and images!


Are you sure you want to proceed? (yes/no):  yes



🗑️ Clearing existing data...
Clearing Firebase collection... ✅
Clearing Cloudinary products folder... ✅

📤 Starting fresh upload process...


Processing: Sugar Free Vanilla syrup: 100%|█| 18/18 [00:38<00:00,  2.13s/i


        Upload Summary        
Total products      : 18
Successful uploads  : 18
Failed uploads     : 0
Success rate       : 100.0%
